In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from pprint import pprint


In [2]:
def vel(m, h, P , P_electric = 250):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eta = 0.95
        
    c0 = P*eta
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))
    c3 = c_Fd
    
    v_max = 30 #km/h
   ### closed form solution of 3-rd degree polynomial

    # p = c1/c3
    # q = -c0/c3
    # print(np.roots([1, 0, p, q]))
    
    ###
    coefs = [c3, 0, c1, -eta*P]
    coefs_electric = [c3, 0, c1, -eta*(P+P_electric)]        
    v = np.max(np.real(3.6*np.roots(coefs)))
    v_electric = np.max(np.real(3.6*np.roots(coefs_electric)))
    if v > 25:
        #print('up-cyclist only')
        return min(v, v_max)
    else:
        #print('up-electric')
        return min(v_electric, 25)


def time_matrix(data_matrix, load_levels,P):
    N = len(data_matrix.index)
    elevation = data_matrix.elevation.to_numpy()
    d_ij_mat = data_matrix.iloc[:, 8:].to_numpy()
    t_ij_mat = np.zeros((N, N, n_levels))
    for i in range(N):
        for j in range(N):
            if i!=j:
                d_ij = d_ij_mat[i, j]
                h = (elevation[j] - elevation[i])/d_ij
                for l in range(n_levels):
                    m = load_levels[l]
                    #print(vel(m,h,P))
                    t_ij_mat[i, j, l] =60*d_ij/vel(m, h, P, P_electric=250)
    return np.round(t_ij_mat,4)

The forces acting on the bicyle are: the aerodynamic drag resistance $F_{D}$, the rolling resistance $F_{R}$ and the component of the gravity force along the direction of motion $F_{G}$:

* $ F_{D} = \frac{\rho C_{D}Av^{2}}{2}$
* $ F_{R} = C_{R}mg cos(arctan(h))$
* $ F_{G} = mg sin(arctan(h))$

We have to distinguish two cases: uphill and downhill.
In uphill, the component of the gravity force along the direction of motion is a resistance force, while in downhill it helps the motion of the bicyle.
so we have, at equilibrium:

* $\mathrm{ \eta P_{traction} = (F_{D} + (F_{R} + F_{G})v }$

where $\eta$ is the efficiency of the mechanic transmission, which is assumed to be 0.95.

If we use a negative h in the formuals above when traversing an edge with negative slope, since $cos(\alpha) = cos(-\alpha)$ and $sin(\alpha) = -sin(-\alpha)$, we have just one expression for the equation to solve.

Moreover, we assume that the bicyle speed is limited in downhill for safety reasons. Within cities, a typical speed limit for for motor vehicles is 50km/h. Considering also that the regulation for pedelec bicyle allows a maximum speed of 25km/h in electric assisted mode, we set a maximum speed in downhill of 30km/h.
The delta energy can be used for recharging the battery for example, similarly to other solutions in the automaotive industry.
The maximum speed limit is imposed on uphill as well.
The strategy for solving the equation is:
* solve the equation for the two cases: (1) $P=P_{cyclist}$, and (2) $P=P_{cyclist}+P_{electric}$ 
* If the cyclist can reach a speed that is higher than the speed allowed in electric mode, use that speed, subject to the limit of maximum speed allowed,
* otherwise use the minimum value among 25 km/h and the solution of problem (2)

In practice we have $P_{electric}$ = 250W, while for $\overline {P_{cyclist}}$ we assume to have few different types of cyclist.

In [3]:
def extract_min(Q,distance):
    minimo=float("inf") #highest python value
    nome=""
    for q in Q: #find the smallest
        if distance[q] <= minimo:
            nome=q
            minimo=distance[q]
    Q.remove(nome) #remove the correct node from the que
    return nome # tell me wich node is removed

def dijkstra(source,weight_matrix,l=0):
    distance=dict() #dict for distance from source
    parent=dict() #dict for parentnes of each node
    nodes=np.array([i for i in range(len(weight_matrix))]) #list of current node to iterate
    for node in nodes:
        distance[node]=float("inf") 
        parent[node]=None
    distance[source]=0
    S=[]
    Q={node for node in nodes} #set used ad que, coupled with distance dict
    while len(Q)>0:
        u=extract_min(Q,distance) #extract the min from Q,using distance dict
        S.append(u) #ultimated nodes
        for v,w in enumerate(weight_matrix[u]): #for each node v, outgoing from u, with weight w
            if distance[v] > distance[u] + w[l]: #relax phase
                distance[v]= distance[u] + w[l]
                parent[v]=u
        #print(f"{u = }")
        #print(f"{distance = }")
    return distance,parent #return distances from source and each node parent !


In [4]:
def shortest_path_matrix(t_ij_mat):
    sp_matrix=np.empty_like(t_ij_mat)
    nodes=np.array([i for i in range(t_ij_mat.shape[0])])
    levels=np.array([i for i in range(t_ij_mat.shape[-1])])
    for source in nodes:
        for l in levels:
            distance,_t=dijkstra(source,t_ij_mat,l)
            sp_matrix[source,:,l]=list(distance.values())
    return sp_matrix



In [5]:
df=pd.read_csv("./instances/small/Fukuoka_01.csv")
N=len(df.index) # customers + depot
N0=N-1 #customers



In [6]:
Q_min = 140
Q_max = 300
n_levels = 4
load_levels = np.array([(0.5+i)*(Q_max-Q_min)/n_levels for i in range(n_levels)])
upper = load_levels+(Q_max-Q_min)/(n_levels*2)
lower = load_levels-(Q_max-Q_min)/(n_levels*2)

In [7]:
s=df.s.to_numpy() #service time
s[0]=0
a=df["tw a"].to_numpy()
b=df["tw b"].to_numpy()
b[0]=np.sum(b[1:])
t_ij_L=time_matrix(df,load_levels,P=150)
t_ij=shortest_path_matrix(t_ij_L)
demand=df.demand.to_numpy()
demand[0]=0
demand=demand/10
Q=150
M_ij=np.empty((N,N)) 
for i in range(N):
    for j in range(N):
        M_ij[i,j]=max([0,b[i]+s[i]+t_ij[i,j,-1]-a[j]])
        #print(f"\t{round(M_ij[i,j],2)}",end=" ")
    #print("")


In [8]:
model = gp.Model("VRPLTT")




Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-17


In [9]:
x_ij=model.addVars( [(i,j) for i in range(N) for j in range(N)],vtype=GRB.BINARY,name="x_ij") #c16
z_ij=model.addMVar( (N,N,n_levels),vtype=GRB.BINARY,name="z_ij") #c18
y_i=model.addVars([i for i in range(N)],vtype=GRB.CONTINUOUS,lb=0,name="y_i") #
f_ij=model.addVars( [(i,j) for i in range(N) for j in range(N)],vtype=GRB.CONTINUOUS,lb=0,name="f_ij") #c17


In [10]:
c8 = model.addConstrs( gp.quicksum( x_ij[i,j] for j in range(0,N) ) == 1
                                         for i in range(1,N))
c9 = model.addConstrs( gp.quicksum( x_ij[j,i] for j in range(0,N) ) == 1
                                         for i in range(1,N))

In [11]:
c10 = model.addConstrs( gp.quicksum( f_ij[j,i]-f_ij[i,j] for j in range(0,N) ) == demand[i] for i in range(1,N) )

In [12]:
c11_l = model.addConstrs( demand[j]*x_ij[i,j] <= f_ij[i,j] for i in range(0,N) for j in range(0,N))
c11_u = model.addConstrs( f_ij[i,j]<= (Q-demand[i])*x_ij[i,j] for i in range(0,N) for j in range(0,N))

In [13]:
c12 = model.addConstrs( gp.quicksum( z_ij[i,j,l] for l in range(n_levels) ) == x_ij[i,j] for i in range(0,N) for j in range(0,N))


In [14]:
c13_l = model.addConstrs( lower @ z_ij[i,j,:] <= f_ij[i,j] for i in range(N) for j in range(N))
c13_u = model.addConstrs( upper @ z_ij[i,j,:] >= f_ij[i,j] for i in range(N) for j in range(N)) 

In [15]:
c14 = model.addConstrs( y_i[i] - y_i[j] + s[i] +t_ij[i,j,:] @ z_ij[i,j,:] <= M_ij[i,j]*(1-x_ij[i,j]) for i in range(0,N) for j in range(1,N) if i != j )

In [16]:
c15_l = model.addConstrs( y_i[i] >= a[i] for i in range(1,N))
c15_u = model.addConstrs( y_i[i] <= b[i] for i in range(1,N))

In [17]:
obj = model.setObjective(gp.quicksum( t_ij[i,j,:] @ z_ij[i,j,:] for i in range(0,N) for j in range(0,N)) ,GRB.MINIMIZE)

In [18]:
model.optimize()


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2705 rows, 2667 columns and 12369 nonzeros
Model fingerprint: 0xaf1be994
Variable types: 462 continuous, 2205 integer (2205 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+03]
  Objective range  [3e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 2e+03]
Presolve removed 618 rows and 568 columns
Presolve time: 0.09s
Presolved: 2087 rows, 2099 columns, 12272 nonzeros
Variable types: 377 continuous, 1722 integer (1722 binary)
Found heuristic solution: objective 119.9679000
Found heuristic solution: objective 55.1635000

Root relaxation: objective 2.610484e+01, 1021 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   26.10484    0   68   55.1635

In [19]:
l=[]
for i in y_i:
    l.append((i,y_i[i].x))
l.sort(key= lambda x:x[1])
l

[(0, 0.0),
 (18, 31.0),
 (12, 61.57389999999998),
 (5, 61.99999999999997),
 (9, 70.99999999999997),
 (6, 72.99999999999997),
 (7, 79.78029999999997),
 (11, 82.17229999999995),
 (4, 85.83629999999997),
 (10, 89.45999999999995),
 (3, 91.16829999999997),
 (8, 95.13199999999995),
 (20, 97.15629999999997),
 (14, 101.99999999999997),
 (15, 111.37399999999992),
 (19, 111.6512),
 (16, 116.80999999999996),
 (2, 117.64),
 (13, 122.99999999999996),
 (1, 123.0),
 (17, 130.99999999999994)]

In [20]:
i=15
j=0
print(f"{x_ij[i,j]=}\n{f_ij[i,j]=}")

x_ij[i,j]=<gurobi.Var x_ij[15,0] (value 0.0)>
f_ij[i,j]=<gurobi.Var f_ij[15,0] (value 0.0)>


In [21]:
z_ij[0,:].x

array([[ 1.,  0.,  0.,  0.],
       [-0., -0., -0.,  0.],
       [-0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [22]:
f_ij

{(0, 0): <gurobi.Var f_ij[0,0] (value 0.0)>,
 (0, 1): <gurobi.Var f_ij[0,1] (value 0.0)>,
 (0, 2): <gurobi.Var f_ij[0,2] (value 0.0)>,
 (0, 3): <gurobi.Var f_ij[0,3] (value 0.0)>,
 (0, 4): <gurobi.Var f_ij[0,4] (value 0.0)>,
 (0, 5): <gurobi.Var f_ij[0,5] (value 0.0)>,
 (0, 6): <gurobi.Var f_ij[0,6] (value 0.0)>,
 (0, 7): <gurobi.Var f_ij[0,7] (value 0.0)>,
 (0, 8): <gurobi.Var f_ij[0,8] (value 0.0)>,
 (0, 9): <gurobi.Var f_ij[0,9] (value 0.0)>,
 (0, 10): <gurobi.Var f_ij[0,10] (value 0.0)>,
 (0, 11): <gurobi.Var f_ij[0,11] (value 0.0)>,
 (0, 12): <gurobi.Var f_ij[0,12] (value 18.400000000000052)>,
 (0, 13): <gurobi.Var f_ij[0,13] (value 0.0)>,
 (0, 14): <gurobi.Var f_ij[0,14] (value 0.0)>,
 (0, 15): <gurobi.Var f_ij[0,15] (value 0.0)>,
 (0, 16): <gurobi.Var f_ij[0,16] (value 0.0)>,
 (0, 17): <gurobi.Var f_ij[0,17] (value 0.0)>,
 (0, 18): <gurobi.Var f_ij[0,18] (value 17.99999999999994)>,
 (0, 19): <gurobi.Var f_ij[0,19] (value 0.0)>,
 (0, 20): <gurobi.Var f_ij[0,20] (value 0.0)>,
 (1,

In [23]:
x_ij

{(0, 0): <gurobi.Var x_ij[0,0] (value 1.0)>,
 (0, 1): <gurobi.Var x_ij[0,1] (value -0.0)>,
 (0, 2): <gurobi.Var x_ij[0,2] (value 0.0)>,
 (0, 3): <gurobi.Var x_ij[0,3] (value 0.0)>,
 (0, 4): <gurobi.Var x_ij[0,4] (value 0.0)>,
 (0, 5): <gurobi.Var x_ij[0,5] (value 0.0)>,
 (0, 6): <gurobi.Var x_ij[0,6] (value 0.0)>,
 (0, 7): <gurobi.Var x_ij[0,7] (value 0.0)>,
 (0, 8): <gurobi.Var x_ij[0,8] (value 0.0)>,
 (0, 9): <gurobi.Var x_ij[0,9] (value -0.0)>,
 (0, 10): <gurobi.Var x_ij[0,10] (value 0.0)>,
 (0, 11): <gurobi.Var x_ij[0,11] (value 0.0)>,
 (0, 12): <gurobi.Var x_ij[0,12] (value 1.0)>,
 (0, 13): <gurobi.Var x_ij[0,13] (value -0.0)>,
 (0, 14): <gurobi.Var x_ij[0,14] (value 0.0)>,
 (0, 15): <gurobi.Var x_ij[0,15] (value 0.0)>,
 (0, 16): <gurobi.Var x_ij[0,16] (value 0.0)>,
 (0, 17): <gurobi.Var x_ij[0,17] (value 0.0)>,
 (0, 18): <gurobi.Var x_ij[0,18] (value 1.0)>,
 (0, 19): <gurobi.Var x_ij[0,19] (value 0.0)>,
 (0, 20): <gurobi.Var x_ij[0,20] (value -0.0)>,
 (1, 0): <gurobi.Var x_ij[1,0